## Tutorial: External Compton scattering
In this tutorial we will show how to compute the Spectral Energy Distribution produced by Compton scattering by the blob electrons of three different photon targets: a Shakura Sunyaev accretion disk, a Broad Line region represented as a spherical shell and a Dust Torus represented as a simple ring.

In [ ]:
# import numpy, astropy and matplotlib for basic functionalities
import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.coordinates import Distance
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
# import agnpy classes
from agnpy.emission_regions import Blob
from agnpy.compton import ExternalCompton
from agnpy.targets import SSDisk, SphericalShellBLR, RingDustTorus
from agnpy.utils.plot import plot_sed, load_mpl_rc

load_mpl_rc()

### Emission Region
In order to provide a cross-check with the literature, in this tutorial we will reproduce the results in Finke (2016) - [see bibliography in the docs](https://agnpy.readthedocs.io/en/latest/bibliography.html). Therefore we adapt an emission region with the same parameters specified in the paper. In this case we consider a broken power-law to describe the electron distribution.

In [ ]:
spectrum_norm = 6e42 * u.erg
parameters = {
    "p1": 2.0,
    "p2": 3.5,
    "gamma_b": 1e4,
    "gamma_min": 20,
    "gamma_max": 5e7,
}
spectrum_dict = {"type": "BrokenPowerLaw", "parameters": parameters}
R_b = 1e16 * u.cm
B = 0.56 * u.G
z = 1
delta_D = 40
Gamma = 40
blob = Blob(R_b, z, delta_D, Gamma, B, spectrum_norm, spectrum_dict)
blob.set_gamma_size(500)
print(f"total number: {blob.N_e_tot:.2e}")
print(f"total energy: {blob.W_e:.2e}")

In [ ]:
print(blob)

In [ ]:
blob.plot_n_e(gamma_power=2)
plt.show()

### External Compton on Shakura Sunyaev Disk
The first result we reproduce is the SED generated by the External Compton (EC) scattering over a Shakura Sunyaev accretion disk, represented by the black lines in Figure 8. Different curves represent different distances between the emission region and the target.

In [ ]:
Image("figures/figure_8_finke_2016.png", width=600, height=400)

Let us start define an accretion disk around a Schwarzschild black hole of mass $1.2 \times 10^9 \, M_{\odot}$ (for an accretion disk around a Schwarzschild BH $\eta=1/12$, $R_{\rm in} = 6 R_g$, where $R_g$ is the gravitational radius).

In [ ]:
# let us adopt the same disk parameters of Finke 2016
M_sun = const.M_sun.cgs
M_BH = 1.2 * 1e9 * M_sun
L_disk = 2 * 1e46 * u.Unit("erg s-1")
eta = 1 / 12
R_in = 6
R_out = 200
disk = SSDisk(M_BH, L_disk, eta, R_in, R_out, R_g_units=True)

In [ ]:
print(disk)

Similarly to the synchrotron self-Compton case, to intitialise the object that will compute the  external Compton (EC) radiation, we simply pass the `Blob` and the `Target` objects to the `ExternalCompton` class. **Note** in this case the distance `r` between the target and the emission region (in `astropy.units`) also has to be specified, as this will modify the SED.

In [ ]:
ec_disk_1 = ExternalCompton(blob, disk, r=1e17 * u.cm)
ec_disk_2 = ExternalCompton(blob, disk, r=1e18 * u.cm)
ec_disk_3 = ExternalCompton(blob, disk, r=1e19 * u.cm)

In [ ]:
# define the array of frequencies over which to calculate the SED
nu = np.logspace(15, 30, 40) * u.Hz

In [ ]:
# compute it
ec_disk_sed_1 = ec_disk_1.sed_flux(nu)
ec_disk_sed_2 = ec_disk_2.sed_flux(nu)
ec_disk_sed_3 = ec_disk_3.sed_flux(nu)

In [ ]:
plot_sed(nu, ec_disk_sed_1, color="k", label=r"$r=10^{17}\,{\rm cm}$")
plot_sed(nu, ec_disk_sed_2, color="crimson", label=r"$r=10^{18}\,{\rm cm}$")
plot_sed(nu, ec_disk_sed_3, color="dodgerblue", label=r"$r=10^{19}\,{\rm cm}$")
# set the same axis limits as in the reference figure
plt.xlim([1e18, 1e29])
plt.ylim([1e-26, 1e-12])
plt.show()

### External Compton Spherical Shell Broad Line Region
The second result we reproduce is the SED generated by the EC scattering over a Broad Line Region (BLR) modelled as an infinitesimally thin spherical shell, of radius $R_{\rm line}$, emitting only a sinlge spectral line, of dimensionless energy $\epsilon_{\rm line} = h \nu_{\rm line} / (m_e c^2)$. A fraction of the disk luminosity $\xi_{\rm line}$ is being reprocessed. The SEDs we aim to reproduce are represented by the black lines in Figure 10, with different curves representing different distances between the emission region and the target. 

In [ ]:
Image("figures/figure_10_finke_2016.png", width=600, height=400)

As in Finke 2016, let us consider a Spherical Shell Broad Line Region emitting the Lyman alpha line and reprocessing 2.4 % of the radiation from the disk defined above.

In [ ]:
# BLR definition
xi_line = 0.024
R_line = 1e17 * u.cm
blr = SphericalShellBLR(L_disk, xi_line, "Lyalpha", R_line)

In [ ]:
ec_blr_1 = ExternalCompton(blob, blr, r=1e16 * u.cm)
ec_blr_2 = ExternalCompton(blob, blr, r=1e18 * u.cm)
ec_blr_3 = ExternalCompton(blob, blr, r=1e19 * u.cm)

In [ ]:
# compute the SEDs
ec_blr_sed_1 = ec_blr_1.sed_flux(nu)
ec_blr_sed_2 = ec_blr_2.sed_flux(nu)
ec_blr_sed_3 = ec_blr_3.sed_flux(nu)

In [ ]:
plot_sed(nu, ec_blr_sed_1, color="k", label=r"$r=10^{17}\,{\rm cm}$")
plot_sed(nu, ec_blr_sed_2, color="crimson", label=r"$r=10^{18}\,{\rm cm}$")
plot_sed(nu, ec_blr_sed_3, color="dodgerblue", label=r"$r=10^{19}\,{\rm cm}$")
# set the same axis limits as in the reference figure
plt.ylim(1e-25, 1e-12)
plt.xlim(1e15, 1e29)
plt.show()

### External Compton Spherical Ring Dust Torus
The second result we reproduce is the SED generated by the EC scattering over a Dust Torus modelled as an infinitesimally thin ring of radius $R_{\rm dt}$. We approximate the target photons energy with the peak energy of the Dust Torus black body ($\epsilon_{\rm dt} = 2.7 k_B T / (m_e c^2)$). The Torus is reprocessing a fraction $\xi_{\rm dt}$ of the disk luminosity. The SEDs we aim to reproduce are represented by the black lines in Figure 11, with different curves representing different distances between the emission region and the target. 

In [ ]:
Image("figures/figure_11_finke_2016.png", width=600, height=400)

In [ ]:
# dust torus definition
T_dt = 1e3 * u.K
csi_dt = 0.1
dt = RingDustTorus(L_disk, csi_dt, T_dt)

In [ ]:
ec_dt_1 = ExternalCompton(blob, dt, r=1e19 * u.cm)
ec_dt_2 = ExternalCompton(blob, dt, r=1e20 * u.cm)
ec_dt_3 = ExternalCompton(blob, dt, r=1e21 * u.cm)

In [ ]:
# compute the SEDs
ec_dt_sed_1 = ec_dt_1.sed_flux(nu)
ec_dt_sed_2 = ec_dt_2.sed_flux(nu)
ec_dt_sed_3 = ec_dt_3.sed_flux(nu)

In [ ]:
plot_sed(nu, ec_dt_sed_1, color="k", label=r"$r=10^{19}\,{\rm cm}$")
plot_sed(nu, ec_dt_sed_2, color="crimson", label=r"$r=10^{20}\,{\rm cm}$")
plot_sed(nu, ec_dt_sed_3, color="dodgerblue", label=r"$r=10^{21}\,{\rm cm}$")
# set the same axis limits as in the reference figure
plt.ylim(1e-28, 1e-14)
plt.xlim(1e16, 1e29)
plt.show()